## **1. Installation**

Update to Tensorflow 2.5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Update CUDA for TF 2.5
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
!dpkg -i libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb
# Check if package has been installed
!ls -l /usr/lib/x86_64-linux-gnu/libcudnn.so.*
# Upgrade Tensorflow
!pip install --upgrade tensorflow==2.5.0

Install Mask R-CNN

In [ ]:
!wget https://pysource.com/extra_files/maskrcnn_colab_demo_commit_17.zip
!unzip maskrcnn_colab_demo_commit_17.zip
import sys
sys.path.append("/content/maskrcnn_colab/mrcnn_demo")
from m_rcnn import *
%matplotlib inline

In [ ]:
!nvidia-smi

## **2. Image Dataset**

Load your annotated dataset


In [ ]:
extract_images(os.path.join("/content/", "/content/drive/MyDrive/DATA/images_val.zip"), "/content/data")

In [ ]:
# Extract Images
images_path = "/content/drive/MyDrive/DATA/images.zip"
annotations_path = "/content/drive/MyDrive/DATA/Bản sao của labels_thainhi.json"

extract_images(os.path.join("/content/",images_path), "/content/dataset")

In [ ]:
dataset_train = load_image_dataset(os.path.join("/content/", annotations_path), "/content/dataset", "train")
dataset_val = load_image_dataset(os.path.join("/content/", annotations_path), "/content/dataset", "val")
class_number = dataset_train.count_classes()
print('Train: %d' % len(dataset_train.image_ids))
print('Validation: %d' % len(dataset_val.image_ids))
print("Classes: {}".format(class_number))

In [ ]:
# Load image samples
display_image_samples(dataset_train)

##**3. Training**

Train Mask RCNN on your custom Dataset.

In [ ]:
# Load Configuration
config = CustomConfig(class_number)
# config.display()
model = load_training_model(config)

In [ ]:
# Start Training
# This operation might take a long time.
train_head(model, dataset_train, dataset_val, config)

##**3.2 Training 2:**

In [ ]:
def load_continue_training_model(config):
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=MODEL_DIR)

    # Which weights to start with?
    init_with = "last"  # imagenet, coco, or last

    if init_with == "imagenet":
        model.load_weights(model.get_imagenet_weights(), by_name=True)
    elif init_with == "coco":
        # Load weights trained on MS COCO, but skip layers that
        # are different due to the different number of classes
        # See README for instructions to download the COCO weights
        print(COCO_MODEL_PATH)
        model.load_weights(COCO_MODEL_PATH, by_name=True,
                           exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",
                                    "mrcnn_bbox", "mrcnn_mask"])
    elif init_with == "last":
        # Load the last model you trained and continue training
        model.load_weights(model.find_last(), by_name=True)

    return model

In [ ]:
model_path = '/content/maskrcnn_colab/logs/object20221130T0229/mask_rcnn_object_0004.h5'
model_1, inference_config = load_inference_model(class_number, model_path)
model_1 = load_training_model(inference_config)

In [ ]:
class CustomDataset_1(utils.Dataset):
    """ Generates a COCO-like dataset, i.e. an image dataset annotated in the style of the COCO dataset.
        See http://cocodataset.org/#home for more information.
    """

    def load_custom_1(self, annotation_json, images_dir, dataset_type="train"):
        """ Load the coco-like dataset from json
        Args:
            annotation_json: The path to the coco annotations json file
            images_dir: The directory holding the images referred to by the json file
        """

        # Load json from file
        print("Annotation json path: ", annotation_json)
        json_file = open(annotation_json)
        coco_json = json.load(json_file)
        json_file.close()


        # Add the class names using the base method from utils.Dataset
        source_name = "coco_like"
        for category in coco_json['categories']:
            class_id = category['id']

            class_name = category['name']
            if class_id < 1:
                print('Error: Class id for "{}" cannot be less than one. (0 is reserved for the background)'.format(
                    class_name))
                return

            self.add_class(source_name, class_id, class_name)

        # Get all annotations
        annotations = {}
        for annotation in coco_json['annotations']:
            image_id = annotation['image_id']
            if image_id not in annotations:
                annotations[image_id] = []
            annotations[image_id].append(annotation)

        # Get all images and add them to the dataset
        seen_images = {}

        # Split the dataset, if train, get 90%, else 10%
        len_images = len(coco_json['images'])
        if dataset_type == "train":
            img_range = [0, len_images]
        else:
            img_range = [0, 0]

        for i in range(img_range[0], img_range[1]):
            image = coco_json['images'][i]
            image_id = image['id']
            if image_id in seen_images:
                print("Warning: Skipping duplicate image id: {}".format(image))
            else:
                seen_images[image_id] = image
                try:
                    image_file_name = image['file_name']
                    image_width = image['width']
                    image_height = image['height']
                except KeyError as key:
                    print("Warning: Skipping image (id: {}) with missing key: {}".format(image_id, key))

                image_path = os.path.abspath(os.path.join(images_dir, image_file_name))
                image_annotations = annotations[image_id]

                # Add the image using the base method from utils.Dataset
                self.add_image(
                    source=source_name,
                    image_id=image_id,
                    path=image_path,
                    width=image_width,
                    height=image_height,
                    annotations=image_annotations
                )

    def load_mask(self, image_id):
        """ Load instance masks for the given image.
        MaskRCNN expects masks in the form of a bitmap [height, width, instances].
        Args:
            image_id: The id of the image to load masks for
        Returns:
            masks: A bool array of shape [height, width, instance count] with
                one mask per instance.
            class_ids: a 1D array of class IDs of the instance masks.
        """
        image_info = self.image_info[image_id]
        annotations = image_info['annotations']
        instance_masks = []
        class_ids = []

        for annotation in annotations:
            class_id = annotation['category_id']
            mask = Image.new('1', (image_info['width'], image_info['height']))
            mask_draw = ImageDraw.ImageDraw(mask, '1')
            for segmentation in annotation['segmentation']:
                mask_draw.polygon(segmentation, fill=1)
                bool_array = np.array(mask) > 0
                instance_masks.append(bool_array)
                class_ids.append(class_id)

        mask = np.dstack(instance_masks)
        class_ids = np.array(class_ids, dtype=np.int32)
        #print("Class_ids, ", class_ids)
        return mask, class_ids

    def count_classes_1(self):
        class_ids = set()
        for image_id in self.image_ids:
            image_info = self.image_info[image_id]
            annotations = image_info['annotations']

            for annotation in annotations:
                class_id = annotation['category_id']
                class_ids.add(class_id)

        class_number = len(class_ids)
        return class_number


In [ ]:
def load_image_dataset_custom(annotation_path, dataset_path, dataset_type):
    dataset_train = CustomDataset_1()
    dataset_train.load_custom_1(annotation_path, dataset_path, dataset_type)
    dataset_train.prepare()
    return dataset_train

In [ ]:
#Extract Images
images_path_val = "/content/drive/MyDrive/DATA/images_val.zip"
annotations_path_val = "/content/drive/MyDrive/DATA/labels_thainhi_val.json"

extract_images(os.path.join("/content/",images_path_val), "/content/dataset_valid")

In [ ]:
dataset_train_1 = load_image_dataset_custom(os.path.join("/content/", annotations_path_val), "/content/dataset_valid", "train")
dataset_val_1 = load_image_dataset_custom(os.path.join("/content/", annotations_path_val), "/content/dataset_valid", "val")
class_number = dataset_train_1.count_classes_1()
print('Train: %d' % len(dataset_train_1.image_ids))
print('Validation: %d' % len(dataset_val_1.image_ids))
print("Classes: {}".format(class_number))

In [ ]:
train_head(model_1, dataset_train_1, dataset_val, config)

##**3.3 Training 3:**

In [ ]:
model_path_2 = '/content/maskrcnn_colab/logs/object20221130T0310/mask_rcnn_object_0004.h5'
model_2, inference_config_2 = load_inference_model(class_number, model_path_2)
model_2 = load_training_model(inference_config_2)

In [ ]:
#Extract Images
images_path_test = "/content/drive/MyDrive/DATA/images_test.zip"
annotations_path_test = "/content/drive/MyDrive/DATA/labels_thainhi_test.json"

extract_images(os.path.join("/content/",images_path_test), "/content/dataset_test")

In [ ]:
dataset_train_2 = load_image_dataset_custom(os.path.join("/content/", annotations_path_test), "/content/dataset_test", "train")
dataset_val_2 = load_image_dataset_custom(os.path.join("/content/", annotations_path_test), "/content/dataset_test", "val")
class_number = dataset_train_2.count_classes_1()
print('Train: %d' % len(dataset_train_2.image_ids))
print('Validation: %d' % len(dataset_val_2.image_ids))
print("Classes: {}".format(class_number))

In [ ]:
train_head(model_2, dataset_train_2, dataset_val, config)

## **4. Detection (test your model on a random image)**

In [ ]:
# Load Test Model
# The latest trained model will be loaded
#test_model, inference_config = load_test_model(class_number)
model_final, inference_config = load_inference_model(class_number, model.find_last())

In [ ]:
test_random_image(model_final, dataset_val, inference_config)

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/DATA/images_N.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/drive/MyDrive/DATA/Datasets')

In [ ]:
!pip install py7zr

In [ ]:
# x MyImageData.zip.001
!7za x /content/drive/MyDrive/DATA/images_N.zip

In [ ]:
import glob
list_path_file = list(glob.glob("/content/drive/MyDrive/DATA/Datasets/images/*"))
list_path_file[:20]

In [ ]:
len(list_path_file)

In [ ]:
def display_instances_custom(image, boxes, masks, class_ids, class_names,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display * \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    colors = colors or random_colors(N, opencv=False)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = colors[i]

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            ax.add_patch(p)

        # Label
        if not captions:
            class_id = class_ids[i]
            score = scores[i] if scores is not None else None
            label = class_names[class_id]
            caption = "{} {:.3f}".format(label, score) if score else label
        else:
            caption = captions[i]
        ax.text(x1, y1 + 8, caption,
                color='w', size=11, backgroundcolor="none")

        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
    ax.imshow(masked_image.astype(np.uint8))
    if auto_show:
        plt.show()
    return masked_image

In [ ]:
def random_colors(N, bright=True, opencv=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 255 if bright else 180
    if opencv is False:
        brightness = 1 if bright else 0.7
    hsv = [(i / N + 1, 1, brightness) for i in range(N + 1)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

In [ ]:
import os
import sys
import random
import itertools
import colorsys
import cv2

import numpy as np
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib import patches,  lines
from matplotlib.patches import Polygon

In [ ]:
def apply_mask(image, mask, color, alpha=0.5):
    """Apply the given mask to the image.
    """
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

In [ ]:
import cv2
for path in list_path_file:
  image = cv2.imread(path)
  try:
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    r = model_final.detect([image])[0]
    img = display_instances_custom(image, r['rois'], r['masks'], r['class_ids'], class_names = 'SS')
  except:
    print(path.split('/')[-1])


In [ ]:
len(img)